# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [15]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [16]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [17]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [18]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [19]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)
    time.sleep(1)


----------------------------------------------------------------------

[0] 市場攤商每天領一片口罩　避免婆媽採買成為疫情缺口

▲口罩國家隊陸續提升產能，助攻口罩日均產能達到1000萬片以上。（圖／記者湯興漢攝）
記者林淑慧／台北報導
公有市場及夜市攤商反映沒空排隊買口罩，經濟部今（20）日宣佈將配送口罩，以每攤位每營業日1片、每片5元，除保障攤商與消費者安全，也為避免日常採買成為疫情缺口。
官員指出，全台參與配售的公有市場及列管夜市共有532處，計54063攤。而除公有市場外，針對地方政府列管之民營傳統市場及列管攤集區攤商，中辦也表示已統計申購口罩意願，預計3月24日起將逐步納入民有市場。
另外，由經濟部斥資加碼的第二批32條新產線，原本表訂將在三月底組裝完成，但在口罩國家隊熱血投入超過2000人次趕工下，今日可望提前完工，並推升口罩日均產能上看1200萬片。
----------------------------------------------------------------------

[1] 解美元荒！韓美簽600億美元貨幣互換協議　史上第2次

▲南韓央行為與Fed簽定貨幣互換協議的9家央行之一。（圖／路透社）
記者王曉敏／綜合報導
南韓央行（BOK）周四表示，已與美國聯準會（Fed）簽訂規模達600億美元的貨幣互換協議（FX Swap），立即向當地金融市場注入美元，以緩解南韓匯市因美元流動性短缺所導致震盪。
此為韓美史上第2次簽訂貨幣互換協議。2008年底金融危機期間，韓美首度簽署了規模達300億美元的協定，起初約定期限為6個月，後延長至2010年2月。
由於南韓當地投資人對美元資金的需求激增，韓元兌換美元匯率，已經貶值至2010年6月來的新低點。稍早，受到韓美簽訂貨幣互換協議激勵，韓元兌美元一度勁揚2.7%，創8年來最大漲幅。據彭博即時匯率，截至稍早，1美元能兌1,245.63韓元。
路透報導，據外匯交易員說法，雙邊貨幣互換協議有機會緩解市場對美元流動性的擔憂，「南韓外匯市場將能從近期的暴跌中喘口氣。」
新冠肺炎疫情重創全球股市，南韓央行周一召開緊急會議，跟隨Fed腳步，決議降息2碼至0.75%，刷新歷史低點，更是南韓央行逾10年來首度於例會外緊急調降利率。周四，南

----------------------------------------------------------------------

[12] 六福客棧改建計劃鳴槍　莊豐如：改建酒店式商用辦公大樓

▲六福旅遊集團董事長莊豐如宣佈，六福客棧提前啟動改建計劃，擬改建為A級商辦大樓。（圖／記者黃克翔攝）
記者林淑慧／台北報導
六福旅遊集團董事長莊豐如今（20）日宣佈，六福客棧提前啟動改建計劃，擬建A級商辦大樓，內部規劃則為酒店式商辦，預計明年第1季開始動工，將成為松江南京商圈亮眼的新指標，集團「資產活用」計畫正式鳴槍。
六福旅遊集團今天表示，將集團100%持有的土地，包含長春段土地、關西鎮土地等，同步於2020年提前啟動興建及開發計畫。莊豐如表示，六福客棧進行拆除後，基地面積約380坪，預計將投入15億元，興建地下5層、地上21層鋼骨結構建築物，總樓地板面積估達6,200坪以上，優先擬朝A級商辦大樓規劃。
目前松江南京商圈的商辦大樓空租率不到5%，並且此區內可開發土地數量較少，周遭建物屋齡多為30至50年的中古辦公大樓，再加上看好台商回流加快將擴大高級商辦需求，未來幾年會出現供給短缺，租賃市場轉為房東市場，在面臨可出租的選擇有限之下，企業租戶也積極採用新租賃策略，如購置量身打造企業大樓，或承租提供更精緻服務的辦公室。
莊豐如說，六福客棧優先擬朝A級商辦大樓規劃，呼應六福旅遊集團追求多元發展的精神，建築物外觀以代表無限可能的弧線做為主體線，也象徵張開手臂擁抱城市的意涵，而立面則以輕盈如鏡的帷幕設計搭配動態的垂直進出面，讓人們站在不同角度都能看見新大樓的不同姿態，在動態的移動過程中翻閱到不同風景。
除了六福客棧改建之外，關西鎮土地達上百公頃，加上既有的動物保育、動物園、主題樂園等優勢，計劃以全新的「六福生態度假村」品牌面世，並打造「樂齡友善社區」，成為樂齡族休閒及退休族享受生活的新天堂。
展望後市，六福客棧改建於今年5月掛照，待主管機關審核通過後，預計明年第1季動工，未來還會持續觀察市場變化，無論出租或出售，皆有優勢及保留彈性，莊豐如表示，六福集團不會停下飯店事業發展腳步，旗下各旅館品牌仍持續拓點，發展區域不僅限於台灣，海外市場也在同步評估中。
------------------------------------------------------------

----------------------------------------------------------------------

[17] 網曝「瑕疵口罩」看過來！經部：藥局、超商日後統一回收

▲口罩國家隊陸續提升產能，助攻口罩日均產能達到1000萬片以上。（圖／記者湯興漢攝）
記者林淑慧／台北報導
口罩國家隊日夜趕工「你累了嗎？」，有藥師在粉專發文，指出近日發現口罩瑕疵逐漸增加，經濟部今（20）日回應表示，已提醒藥局、超商等通路，瑕疵口罩先不出售，網路預購的民眾也能到超商換貨，日後會再統一回收。
有藥師在《北大藥局》粉專PO文指出，近幾天發現口罩瑕疵有增加趨勢，除了鋼條脫落、繩子貼反、還有耳掛線斷及出現髒汙的情況，藥師無奈地說，直接少掉八包的量，「瑕疵品會影響到每天實際能發放的數量，請大家多包容」。
對此，經濟部工業局官員指出，工業局在每一個口罩工廠均有駐廠人員，已經要求駐廠同仁轉達口罩生產者，應注意品質的管控，也謝謝藥房藥師在第一缐的辛苦。
另外，由經濟部斥資加碼的第二批32條新產線，原本表訂將在三月底組裝完成，但在口罩國家隊熱血投入超過2000人次趕工下，今日可望提前完工，並推升口罩日均產能上看1200萬片。
----------------------------------------------------------------------

[18] 台股強勢攻上9200點！大立光、聯發科攻漲停　指數暴衝570點

▲ 中央銀行、金管會及國安基金救市三支箭齊發，台股今日一掃陰霾應聲大漲，盤中漲點570點，收復9200點關卡。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台股今（20）日開盤後一路飆升，連續收復9000、9100與9200點關卡，漲點擴大到500點以上，目前來到9251點、上漲570點。
股王大立光（3008）強勢攻漲停，目前來到3575元；聯發科（2454）更是自10點左右就在漲停板附近狹幅震盪，目前也維持漲停價位，為301元。
另外，台積電（2330）目前亦強漲8.67%、來到270元；鴻海（2317）也收復70元大關，目前在70.9元、上漲6.49%
看到漲跌家數，目前上市總共有127家亮燈漲停，763家維持紅盤；28家下跌，跌停則為3家。
--------------------------------------

----------------------------------------------------------------------

[30] 東森購物向台灣隊長致敬　不計成本捐贈超暖心千萬防疫禮

▲東森購物捐贈防疫物資，向辛苦的警消醫護、疫苗研發團隊與其家人致敬。（圖／東森購物提供）
記者洪菱鞠／台北報導
為了向守護全民健康的「台灣隊長」包括醫護警消、疫苗研發團隊與其家人致敬，東森購物電話行銷團隊精選目前通路最暢銷的保健品和保養品，邀請供應商共襄盛舉，不計成本全數捐贈予第一線醫護人員，希望能為他們及家人補充健康好體力、維持美麗好氣色，而此次贊助物資總金額高達新台幣1,216萬元。
東森購物表示，此次捐贈防疫物資包括東森自然美贊助的「自然美－瑞昇超逆齡賦活膠囊」及「東森自然美瑞昇超逆齡賦活膠囊」，東森自然美執行長林燕玲觀察到，女性醫護人員因長時間戴口罩，導致臉部肌膚變得容易乾癢、脫皮或泛紅，希望她們在辛苦工作之餘，也能收到讓肌膚保持最佳狀況的優質MIT保養產品，自然散發美麗好氣色。
在新冠肺炎疫情影響下，民眾降低出門頻率，電商平台此時成為購物安全考量首選，東森購物電話行銷執行長紀坤傑指出，從疫情爆發開始，抗疫產品和保健食品等需求都非常大。東森購物始終秉持顧客至上的服務精神，除持續關心疫情現況，也積極配合政府政策，疫情期間因國內農漁產外銷受到衝擊，因此率先響應農委會協助開賣鳳梨釋迦、午仔魚與石斑魚力挺小農，另外也攜手新北市政府推出「新北專區」，幫業者拓展虛擬通路商機，更不斷精進服務品質，為台灣社會盡一份心力，也持續為所有會員帶來最愉悅舒適的購物體驗。
▲▼疫情期間，東森購物率先響應農委會政策，於旗下電視購物與購物網等通路大力促銷國內優質農漁產品，台東鳳梨釋迦一開賣立即狂銷3萬台斤，午仔魚與石斑魚2小時熱賣8,000條。（上圖／記者徐文彬攝、下圖／東森購物提供）
----------------------------------------------------------------------

[31] 陸84艘輪船赴沙國搶原油　網嘆：像雙11...真相出爐了

▲國際油價持續下跌。（圖／路透社）
記者曾俊豪／綜合報導
大陸網站近日流傳一篇文章，表示有84艘中國輪船啟航，前往沙國購買原油。不過，文章經媒體查證，並非事實。業內人士分析，國際油價暴跌之

▲美國油價今（20）日飆漲近24%，從18年低點回升，創史上最大單日漲幅。（圖／達志影像／美聯社）
記者林淑慧／台北報導
沙烏地阿拉伯與俄羅斯兩大產油國增產競爭，近期國際油價波動劇烈，美國總統川普考慮介入調停，激勵美國油價今（20）日飆漲近24%，從18年低點回升，創史上最大單日漲幅。
新冠肺炎疫情在全球持續蔓延，儘管各主要經濟體宣示將推出刺激經濟措施，卻仍無法緩解因疫情大流行及價格戰引起的擔憂，紐約西德州中級原油（WTI）價格昨天暴跌24.4%，創1991年波灣戰爭爆發以來第2大單日跌幅，收盤價創2002年2月以來新低。
為緩和低油價對美國業者的衝擊，美國能源部宣佈將招標採購3000萬桶原油作為戰略儲備，外傳美國總統川普考慮介入兩大產油國油價戰調停，透過外交施壓沙國減產，並威脅對俄羅斯實施制裁，以利恢復油價穩定，激勵美國油價止跌回升。
WTI今天飆漲4.85美元，漲幅23.8%，收在每桶25.22美元，創史上最大單日漲幅。倫敦布倫特油價則上漲3.59美元，漲幅14.4%，收在每桶28.47美元。
國際油價走勢近期波動劇烈，高盛集團（Goldman Sachs）已大幅下修今年第二季國際油價預測，預估WTI與布倫特每桶均價只剩20美元。在新冠肺炎疫情延燒下，高盛預估至三月底，全球每日原油消耗量恐銳減800萬至900萬桶。
----------------------------------------------------------------------

[42] 紐約油價激漲24％「史上最猛」！川普買3千萬桶原油奏效　美考慮介入石油戰

▲美國頁岩油有一定市佔率，沙烏地對俄羅斯打石油戰也會連帶影響美國，川普已考慮介入油市。（圖／路透社）
中央社／記者尹俊傑
美國油價今天飆漲近24%，從18年低點回升，創史上最大單日漲幅。面對沙烏地阿拉伯與俄羅斯增產競爭，美國總統川普考慮介入調停，協助油價擺脫昨天大跌陰影。
在2019年冠狀病毒疾病（COVID-19，武漢肺炎）疫情蔓延、沙烏地阿拉伯與俄羅斯油價戰雙重打擊下，國際油價近期走勢淒慘。紐約西德州中級原油（WTI）價格昨天暴跌24.4%，創1991年波灣戰爭爆發以來第2大單日跌幅，收盤價創2002年2月以來新低。
為緩和低油價對美國業者的衝擊，美國能源部今天奉川普指示，宣布將招標採購3000萬桶原油作為戰略儲備，刺激

----------------------------------------------------------------------

[53] 疫情升溫！證期局加強出國控管　券商員工返台要自主隔離

▲金管會證期局今天宣布，要求證券商應強化相關防疫措施。（圖／資料照）
記者紀佳妘／台北報導
新冠肺炎（COVID-19）疫情在各國蔓延，台灣確診案例累計破百，繼金管會保險局祭出3項出國加強管控措施，證期局今（19）日也宣布，要求證券商應強化相關防疫措施，應掌握員工出國狀況，若回台應依規定進行居家檢疫或自主健康管理。
因應疫情發展，證期局表示，要求證券商應再強化相關防疫措施，包括增加要求業者應宣導員工及各級主管應避免非必要的公務及非公務出國，並應掌握公司人員出國狀況及採行相關防疫管理措施。
證期局指出，若有人員自國外返台，應確實依規定要求進行居家檢疫或自主健康管理。
為提升證券商重視防疫及落實執行，證期局說，17日要求各證券商應在20日前通報各公司的防疲計畫與執行情形，同時也應在證期局每周的疫情調查表中，增加填報公司人員出國狀況及防疫控管資訊。
這份疫情調查表內容，包括查填已出國及未來一定期間預計出國的人數、職務、出國地區、出國期問、回國後執行防疫措施等内容，以利證期局掌握資訊及督促業者落實控管執行。
此外，保險局在16日率先對保險業祭出3項出國加強管控措施，董事長及總經理若有出國規劃，需事先通報，同時也要求保險公司設置「防疫長」，且層級拉高至總經理，並成立「防疫長群組」督促公司確實落實防疫工作。
----------------------------------------------------------------------

[54] 央行經濟成長預期未「保2」　祭2000億融通方案護中小企業

▲ 央行總裁楊金龍。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
全球受到新冠肺炎(COVID-19)疫情干擾，衝擊供應鏈與最終需求，央行今（19）日舉行理監事會議，預估全年經濟成長率未「保2」，為1.72%。
央行表示，隨著肺炎疫情擴散，預期今年國際景氣明顯趨緩，不利台灣出口，加上來台旅客劇減，抑制服務輸出成長。內需方面，疫情影響國內消費信心及就業市場，預期民間消費支出保守；另外，廠商投資計畫亦可能受影響，且去(2019)年資本設備進口大幅擴增，墊高比較基

----------------------------------------------------------------------

[64] 台積電染疫效應！聯電禁外國訪客入廠、智邦架帳篷、南茂租貨櫃屋戶外開會

▲聯電今日內部公告，所有外國訪客一律禁止進入廠區。（圖／記者周康玉攝）
記者周康玉／新竹報導
台積電（2330）昨（18）日公告一名員工確診新冠肺炎（COVID-19）後，明（20）日將開始實施分流上班，竹科大廠為防疫出奇招，聯電今（19）日宣布，禁止所有外國訪客進入公司；半導體封測廠南茂設置貨櫃屋，讓客戶或供應商可「戶外開會」；網通廠智邦則是在門口搭遮雨棚架，接待訪客；竹科館領局也帶頭示範，大型會議採梅花座。
根據聯電（2303）內部文件指出，因應疾管署昨（18）日最新宣布因國際疫情持續升溫，境外移入個案暴增，今日凌晨零時起，非本國籍人員一律限制入境。不分國籍，所有入境者一律居家檢疫14天。
聯電表示，不僅所有國外訪客不准進入公司，相關裝機、維修等行程盡量不做安排，先確保公司員工健康安全為優先。至於分流辦公，聯電表示，分區辦公會是下一階段的措施，一旦政府宣布有社區感染，就馬上啟動該機制。
由於竹科染疫升溫，竹科管理局帶頭做示範，像是記者會或大型會議地點皆改到通風良好、有窗戶、較為空曠的會議室，座位也採梅花座。
封測廠南茂（8150）目前禁止供應商和客進入廠區；針對內部員工，也減少因公出差。針對訪客，南茂在門口承租貨櫃屋作為替代方案，作為臨時接待訪客區，由各部門預約時段，該時段使用完畢就清潔消毒。
網通大廠智邦（2345）則在廠區入口架起戶外帳篷，並擺設桌椅，讓客戶、供應商等訪客不需進入廠區，即可在戶外訪客區洽談相關事宜。
▼竹科大廠防疫出奇招，南茂承租貨櫃屋戶外開會，下同。（圖／記者周康玉攝）
▼竹科大廠防疫出奇招，智邦科技架外棚會客。（圖／記者周康玉攝）
----------------------------------------------------------------------

[65] 財訊／「中國式復工」　一齣上下呼攏的荒唐劇

文／蔣清平
「快快快，把燈全打開，車間的馬達轟轟轟聲音要出來！」三月初的杭州蕭山經濟技術開發區一名台商對著廠區領班大喊。他邊喘氣邊無奈地說，中央聯合檢查組來稽查復工情況，現在查得嚴，「我們復工

----------------------------------------------------------------------

[69] 理財周刊／步步為營　不搶短

文／洪寶山
雪崩式大跌後市場信心不容易立刻恢復，投資人千萬不要隨便心動手癢見獵心喜，記取歷史教訓等盤勢落底確實止穩後再出手，才能反敗為勝。
將來後人寫近二十年的金融史，必然會提到三次重要的股市牛熊分水嶺：二○○一年的九一一恐怖攻擊事件、二○○八年金融海嘯、二○二○年COVID-19新冠病毒。三大利空都造成股市大跌。道瓊指數打噴嚏，全球股市跟著跳水也跌入技術性熊市。逼得美國聯準會短短十三天之內連降六碼，直接零利率，川普更推出一兆美元的財政政策與啟動「直升機撒錢」。
利多一波又一波，多頭的中期反彈要來了嗎？以古鑑今，回顧九一一恐怖攻擊是美國本土首度受到攻擊，重創美國人的安全感，當時正處在聯準會降息週期，事發後聯準會降息兩碼，並持續降息至二○○三年七月的一％。二○○八年的金融海嘯不僅零利率，更三度量化寬鬆。九一一恐怖事件降息二十二碼，金融海嘯降息二十一碼，外加三次量化寬鬆(相當於六碼)，那麼這次瘟疫讓聯準會已經降息六碼，直接歸零，而鮑爾還強調美國不適合負利率。
市場預期央行之間的降息梭哈啟動，金融市場陷入大混亂，投資人採取現金為王的策略將造成金融商品流動性劇降，在新秩序還沒重建之前，單日震盪幅度擴大，以為抄底機會來臨，結果進場才知道是死在半山腰。當下投資人要思考的是，不止損，自己的口袋夠不夠深？止損，能不能管住自己容易衝動的心，遠離盤面不搶短？
這次疫情初期全球以為武漢封城就阻止了新冠病毒擴散，卻選擇性的遺忘武漢封城前一晚三十萬人大逃離的危險，雖然台灣擋下了，日韓歐美卻先後在一個月後成為重災區，封城已成為新常態，經濟活動瞬間劇停。各國央行會大動作降息，不外乎今年全球經濟衰退機率高達九○％，而且在疫苗研發出來之前，經濟衰退還會持續，全球供應鏈與消費力瞬間急煞車，將來重新開機也需要時間。
美股十一年的牛市僅用十九個交易日就結束，這波來得又急又快的熊市，考驗高負債企業的存活能力，也考驗高槓桿投資人的風險控制能力，期指斷頭之後就是融資斷頭，接著就是企業暴雷，海水退潮後才知誰在裸泳。
-----------------------------------------------------------

----------------------------------------------------------------------

[76] 台股重挫創3年半新低　金管會出手救市「禁止平盤以下放空」

▲台股今天大跌逾500點，失守8700點，再創3年多新低。（圖／記者湯興漢攝）
記者紀佳妘／台北報導
新冠肺炎（COVID-19）疫情升溫，全球金融市場震盪劇烈，金管會今（19）日宣布，即日起當日收盤價跌幅達3.5%以上，隔天交易日不得以低於前一交易日收盤價放空。
為維持證券交易市場的秩序及穩定，保障投資人權益，避免個股價格因放空交易行為加速跌勢，金管會宣布即日起採取2措施，首先是投資人融券賣出或向證券交易所借券系統、證券商或證券金融事業借券賣出上市櫃股票及台灣存託憑證，於該有價證券當日收盤價跌幅達3.5%以上，次一交易日不得以低於前一交易日收盤價融券及借券賣出。
不過，次一交易日的收盤價跌幅仍達3.5%以上，持續進行價格限制措施，但證券商及期貨商因辦理業務，避險需求所為的融券及借券賣出，則不在此限。
金管會證期局副局長張振山指出，過去在2015年6月1日漲跌幅從7%放寬至10%，當時規定跌停10%，隔天不能平盤下放空，而2015年8月當時祭出的救市措施則是不論股票漲跌多少，一律禁止平盤下放空。
以今天來看，上市743檔、上櫃490檔，總計1,233檔跌幅超過3.5%，換句話說，明天起若這1,233檔就禁止平盤下放空，這也是金管會首度「有條件」禁止平盤下放空。
金管會也限制不得以低於前一交易日收盤價融券及借券賣出的交易日，投資人從事有價證券當日沖銷交易，現券賣出後未完成反向交易，賣出價格低於前一交易日收盤價者，不得變更交易類別為融券賣出或借券賣出。
張振山說明，投資人在做當沖交易者，若是採用「先賣後買」，因此賣掉後券卻買不回來的時候，一律禁止做融券，只能以借券等方式，但這種方式通常成本會比較高。根據統計，到2月底為止，融資餘額達1,046.65億元，融券張數達56萬1,779張。
----------------------------------------------------------------------

[77] 央行降息1碼　房貸族每千萬元「月省利息1166元」！

▲房市。（圖／記者李毓康攝）
記者吳靜君、李瑞瑾／台北報導
央行宣布降

----------------------------------------------------------------------

[84] 台指期大跌566點　分析師：8800點換倉缺口難回補、震盪整理機會大

▲台指期3月19日走勢。（圖／取自期交所）
記者余弦妙／台北報導
美股持續暴跌觸發熔斷，台指期今（19）日也受到驚嚇，開盤下跌99點，灌破8900點來到8820點，接著一路走跌，盤中賣壓湧現一度來到8268點，大跌651點，不過隨即止跌，回到8400點附近震盪整理，終場以8353點作收，下跌566點，與現貨相較，逆價差328.34點。
展望後市，群益期貨分析師容逸燊分析，今日盤中出現恐慌性賣壓，不過隨即受到陸股反彈激勵，隨即止穩小幅反彈，接下來的走勢應該會在8800點周邊震盪。
容逸燊表示，今天盤中其實有出現恐慌性的賣壓，許多上市櫃公司都達到跌停板，不過下午陸股A50指數出現反彈，甚至一度反彈到平盤附近，再加上國安基金下午也要開會，基於這兩個原因也讓台指在尾盤不再下殺，甚至在晚間電子盤出現大幅反彈。
國安基金是否真的進場，目前還是未知數，但容逸燊說，至少可以確定的是，國安基金召開會議這項舉動，就是希望可以穩定散戶信心，不讓大家繼續恐慌。
----------------------------------------------------------------------

[85] 元大期營運再成長　發放現金股利3.2元創歷史新高

▲元大期貨股利創歷史新高。（圖／元大期貨提供）
記者余弦妙／台北報導
元大期貨(6023)今(19)日董事會決議通過配發現金股利3.2元，創歷史新高紀錄，較前年成長14.2%，以近日收盤價計算殖利率達6.6%，本次配息率超過稅後獲利的70%。元大期貨108年公司營運動能持續成長，稅前淨利達14.1億元、年成長13.25%、稅後EPS為4.89元，成為台灣首家且連續三年站穩稅前獲利十億元大關的期貨商。
對此，元大期貨總經理周筱玲表示，元大期貨為台灣期貨商的龍頭，國內外期權市佔率、獲利能力等各項數據皆穩居第一，可交易多達16家國際交易所商品，範圍涵蓋指數、匯率、能源、金屬等八大類期權商品，透過24小時全年無休的服務團隊，提供投資人跨國零時差的專業交易服務。
此外，元大期貨表示，將致力於發展金融創新，持續滿足各類型